# Анализ текстов

Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм" (инструкция приведена выше). Обратите внимание, что загрузка данных может занять несколько минут

In [126]:
import sklearn
import pandas as pd
import sklearn.svm
from sklearn.svm import SVC
import numpy as np
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [87]:
newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )

Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.

In [88]:
X_train = newsgroups.data

In [89]:
y_train = newsgroups.target

In [90]:
vector = TfidfVectorizer()
Matrix=vector.fit_transform(X_train)
#Y=vector.fit_transform(y_train)

In [92]:
#матрица текст-слово(TF-IDF)
Matrix

<1786x28382 sparse matrix of type '<class 'numpy.float64'>'
	with 303138 stored elements in Compressed Sparse Row format>

In [93]:
#здесь мы узнаем названия колонок, то есть всех слов, встречающихся втексте
#каждая колонка-отдельный признак текста, то есть отдельное слово
words = vector.get_feature_names()

Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).

In [99]:
grid = {'C': np.power(10.0, np.arange(-5, 6))} #создали массив С
#для кросс-валидации
cv = KFold(n_splits=5, shuffle=True, random_state=241)
#метод опорных векторов
clf = sklearn.svm.SVC(kernel='linear', random_state=241)
#какому алгоритму, какой параметр подбираем 
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
#обучаем
gs.fit(Matrix, y_train)

AttributeError: 'GridSearchCV' object has no attribute 'grid_scores_'

In [105]:
C = gs.best_params_['C']

In [106]:
C

1.0

Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.

In [110]:
clf1 = sklearn.svm.SVC(C=1.0, kernel='linear', random_state=241)

In [111]:
clf1.fit(Matrix, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=241,
    shrinking=True, tol=0.001, verbose=False)

Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.

In [116]:
coef = clf1.coef_

In [120]:
#приводим разреженную матрицу к обычной
coefs = abs(clf1.coef_.todense().A1)
coefs1 = np.argsort(coefs)

In [121]:
coefs1

array([ 7801, 21437,  9144, ...,  5088, 12871, 24019])

In [131]:
most_important_words_indexes = np.argsort(abs(clf1.coef_.toarray()[0]))[-10:]

In [132]:
most_important_words_indexes

array([22936, 15606,  5776, 21850, 23673, 17802,  5093,  5088, 12871,
       24019])

In [133]:
most_important_words = np.array(vector.get_feature_names())[most_important_words_indexes]

In [134]:
most_important_words

array(['sci', 'keith', 'bible', 'religion', 'sky', 'moon', 'atheists',
       'atheism', 'god', 'space'], dtype='<U80')

In [135]:
most_important_words_sorted = sorted(most_important_words)

In [136]:
most_important_words_sorted 

['atheism',
 'atheists',
 'bible',
 'god',
 'keith',
 'moon',
 'religion',
 'sci',
 'sky',
 'space']

In [137]:
resulting_string = " ".join(most_important_words_sorted)

file_answer = open("answer.txt", "w")
file_answer.write(resulting_string)
file_answer.close()